In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import neptune # for logging progress and results
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
from attrs import define, field, fields, Factory

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType

from pyphocorehelpers.print_helpers import generate_html_string

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot, create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions # Add session indicators to pyqtgraph plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions
import pyphoplacecellanalysis.External.pyqtgraph as pg
# 'time_bin_indices': valid_time_bin_indicies, 'posterior_to_pf_mean_surprise': one_left_out_posterior_to_pf_surprises_mean, 'posterior_to_scrambled_pf_mean_surprise': one_left_out_posterior_to_scrambled_pf_surprises_mean}
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_expected_vs_observed_firing_rates
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_surprise_difference_plot

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import _update_pipeline_missing_preprocessing_parameters
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

from scripts.run_BatchAnalysis import post_compute_validate, _on_complete_success_execution_session

session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

good_contexts_list = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54')
]

from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass




In [3]:
%pdb off
# %%viztracer
# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE


local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True


## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.


    # ==================================================================================================================== #
    # Load Pipeline                                                                                                        #
    # ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=[#'_perform_estimated_epochs_computation',  # AL:WAYS OFF
                                            '_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation', # AL:WAYS OFF
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation' # AL:WAYS OFF
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding' # AL:WAYS OFF
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']

curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'


if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        print(f'cannot load global results: {e}')

# 6m 1.1s
# 12m 15.6s


Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\loadedSessPickle.pkl... done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\loadedSessPickle.pkl.
property already present in pickled version. No need to save.
using provided computation_functions_name_includelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definit

In [4]:
%matplotlib qt
from neuropy.utils.matplotlib_helpers import interactive_select_grid_bin_bounds_2D
# fig, ax, rect_selector, set_extents = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True)

grid_bin_bounds = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True, should_apply_updates_to_pipeline=False)
print(f'grid_bin_bounds: {grid_bin_bounds}')
print(f"Add this to `specific_session_override_dict`:\n\n{curr_active_pipeline.get_session_context().get_initialization_code_string()}:dict(grid_bin_bounds=({(grid_bin_bounds[0], grid_bin_bounds[1]), (grid_bin_bounds[2], grid_bin_bounds[3])})),\n")

blocking and waiting for user input. Press [enter] to confirm selection change or [esc] to revert with no change.
user confirmed extents: (22.397021260868584, 245.3970212608686, 133.66465594522782, 155.97244934208123)
Add this to `specific_session_override_dict`:

IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'):dict(grid_bin_bounds=(((22.397021260868584, 245.3970212608686), (133.66465594522782, 155.97244934208123)))),

grid_bin_bounds: (22.397021260868584, 245.3970212608686, 133.66465594522782, 155.97244934208123)
Add this to `specific_session_override_dict`:

IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'):dict(grid_bin_bounds=(((22.397021260868584, 245.3970212608686), (133.66465594522782, 155.97244934208123)))),



In [4]:
## Post Compute Validate 2023-05-16:
from scripts.run_BatchAnalysis import post_compute_validate

post_compute_validate(curr_active_pipeline)

were pipeline preprocessing parameters missing and updated?: False


In [5]:
curr_active_pipeline.reload_default_computation_functions()
extended_computations_include_includelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'] # do only specifiedl , 'long_short_rate_remapping'
force_recompute_global = force_reload
# force_recompute_global = True
# extended_computations_include_includelist=['long_short_post_decoding'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if len(newly_computed_values) > 0:
	print(f'newly_computed_values: {newly_computed_values}. Saving global results...')
	try:
		# curr_active_pipeline.global_computation_results.persist_time = datetime.now()
		# Try to write out the global computation function results:
		curr_active_pipeline.save_global_computation_results()
	except Exception as e:
		print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
		print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
	print(f'no changes in global results.')
# 10m 29.5s

included includelist is specified: ['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze"
jonathan_firing_rate_analysis already computed.
long_short_fr_indicies_analyses already computed.
long_short_decoding_analyses already computed.
long_short_post_decoding already computed.
done with all batch_extended_computations(...).
no changes in global results.


In [7]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True)


https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/
https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-151
shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
long_only_aclus: [12, 21, 48]
short_only_aclus: [18, 19, 65]
shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
long_only_aclus: [12, 21, 48]
short_only_aclus: [18, 19, 65]
include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-06-20\kdiba_gor01_two_2006-6-07_

on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =12
	selection_indicies: [12]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =13
	selection_indicies: [12 13]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =15
	selection_indicies: [12 13 15]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =17
	selection_indicies: [12 13 15 17]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =20
	selection_indicies: [12 13 15 17 20]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =24
	selection_indicies: [12 13 15 17 20 24]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =5
	selection_indicies: [5]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =13
	selection_indicies: [ 5 13]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =15
	selection_indicies: [ 5 13 15]
on_selected_epochs_changed(...)
current

### Pipeline Loading/Saving [Optional]

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
# TypeError: cannot pickle 'MplMultiTab' object
# PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell
# TypeError: cannot pickle 'PyQt5.QtCore.pyqtSignal' object

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)

In [ ]:
curr_active_pipeline.save_global_computation_results() # PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell

In [ ]:
curr_active_pipeline.load_pickled_global_computation_results()

In [ ]:
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
curr_active_pipeline.save_global_computation_results()

# End Run

In [6]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())

# 2023-05-19 - Testing S-only emergence, L-only replays in S, peak position remappings 

In [7]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df.copy()
neuron_replay_stats_df

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
2,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,8.512636,9.008943,0.496307,15.353845,16.269302,0.915458,11.933241,12.639123,0.705882,0,285,104,181,NeuronType.CONTAMINATED
3,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,0.115619,0.588317,0.472698,7.149594,10.001891,2.852298,3.632606,5.295104,1.662498,1,29,1,28,NeuronType.INTERNEURONS
4,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,30.815391,23.468679,-7.346712,85.443885,63.301453,-22.142432,58.129638,43.385066,-14.744572,2,384,145,239,NeuronType.CONTAMINATED
5,99.397021,True,121.397021,True,False,SplitPartitionMembership.SHARED,0.393321,0.421610,0.028288,9.225890,10.241202,1.015313,4.809606,5.331406,0.521800,3,10,2,8,NeuronType.PYRAMIDAL
6,213.397021,True,169.397021,True,False,SplitPartitionMembership.SHARED,1.313865,0.960528,-0.353337,14.421421,15.662375,1.240953,7.867643,8.311451,0.443808,4,210,82,128,NeuronType.PYRAMIDAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,0.954859,0.809236,-0.145623,14.435234,14.621707,0.186473,7.695047,7.715472,0.020425,59,118,55,63,NeuronType.CONTAMINATED
62,147.397021,True,141.397021,True,False,SplitPartitionMembership.SHARED,0.845354,1.011254,0.165900,9.354331,9.210371,-0.143959,5.099842,5.110813,0.010971,60,105,29,76,NeuronType.PYRAMIDAL
63,47.397021,True,85.397021,True,False,SplitPartitionMembership.SHARED,1.708625,0.727221,-0.981404,14.368023,16.532061,2.164038,8.038324,8.629641,0.591317,61,135,51,84,NeuronType.PYRAMIDAL
64,233.397021,True,103.397021,True,False,SplitPartitionMembership.SHARED,2.825995,5.278539,2.452543,19.461425,19.292415,-0.169010,11.143710,12.285477,1.141766,62,217,84,133,NeuronType.PYRAMIDAL


In [8]:
neuron_replay_stats_df.sort_values(by=['long_pf_peak_x'], inplace=False, ascending=True)

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
12,23.397021,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.391955,0.256217,-0.135738,8.701443,10.519667,1.818224,4.546699,5.387942,0.841243,10,4,1,3,NeuronType.PYRAMIDAL
41,45.397021,True,83.397021,True,False,SplitPartitionMembership.SHARED,0.862529,0.915665,0.053136,11.802005,12.818683,1.016677,6.332267,6.867174,0.534906,39,81,27,54,NeuronType.PYRAMIDAL
63,47.397021,True,85.397021,True,False,SplitPartitionMembership.SHARED,1.708625,0.727221,-0.981404,14.368023,16.532061,2.164038,8.038324,8.629641,0.591317,61,135,51,84,NeuronType.PYRAMIDAL
20,49.397021,True,63.397021,True,False,SplitPartitionMembership.SHARED,2.768203,2.742977,-0.025226,13.309307,14.252601,0.943293,8.038755,8.497789,0.459034,18,197,61,136,NeuronType.PYRAMIDAL
15,53.397021,True,219.397021,True,False,SplitPartitionMembership.SHARED,1.362429,1.440596,0.078168,8.931578,8.190237,-0.741341,5.147003,4.815417,-0.331587,13,54,28,26,NeuronType.PYRAMIDAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,19.376079,14.758426,-4.617653,70.520662,56.224553,-14.296109,44.948370,35.491490,-9.456881,55,384,145,239,NeuronType.CONTAMINATED
58,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,4.110594,2.897432,-1.213162,17.634116,17.436274,-0.197842,10.872355,10.166853,-0.705502,56,307,112,195,NeuronType.CONTAMINATED
59,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,23.295877,22.842900,-0.452977,59.392242,77.728279,18.336037,41.344059,50.285590,8.941530,57,384,145,239,NeuronType.CONTAMINATED
61,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,0.954859,0.809236,-0.145623,14.435234,14.621707,0.186473,7.695047,7.715472,0.020425,59,118,55,63,NeuronType.CONTAMINATED


In [9]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership, SetPartition
@define(slots=False)
class TrackPartitioningResult:
	is_S_pf_only: np.ndarray
	is_S_only: np.ndarray
	S_only_aclus: np.ndarray
	S_only_df: pd.DataFrame

# Four distinct subgroups are formed:  pf on neither, pf on both, pf on only long, pf on only short
# L_only_aclus, S_only_aclus

In [10]:
## 2023-05-19 - Get S-only pfs
is_S_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_long_pf']), neuron_replay_stats_df['has_short_pf'])
is_S_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.RIGHT_ONLY
assert (is_S_pf_only == is_S_only).all()
S_only_aclus = neuron_replay_stats_df.index[is_S_only].to_numpy()
neuron_replay_stats_df[is_S_pf_only]

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
18,NaN,False,125.397021,True,True,SplitPartitionMembership.RIGHT_ONLY,0.432887,0.287354,-0.145533,8.638089,8.271821,-0.366268,4.535488,4.279587,-0.25590,16,35,6,29,NeuronType.PYRAMIDAL
19,NaN,False,103.397021,True,True,SplitPartitionMembership.RIGHT_ONLY,0.153454,1.071407,0.917953,NaN,19.691029,NaN,NaN,10.381218,NaN,17,67,0,67,NeuronType.PYRAMIDAL
65,NaN,False,101.397021,True,True,SplitPartitionMembership.RIGHT_ONLY,0.193028,0.456906,0.263878,7.398566,6.006789,-1.391778,3.795797,3.231847,-0.56395,63,28,8,20,NeuronType.PYRAMIDAL


In [11]:
## Show L-only pfs stop replaying on S
is_L_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_short_pf']), neuron_replay_stats_df['has_long_pf'])
is_L_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.LEFT_ONLY
assert (is_L_pf_only == is_L_only).all()
L_only_aclus = neuron_replay_stats_df.index[is_L_only].to_numpy()
neuron_replay_stats_df[is_L_only]


## For ('kdiba', 'gor01', 'one', '2006-6-09_1-22-43') - Have L-only cells [24, 98] that have ['short_num_replays'] = [8, 7]. We were hoping that there would be few to no replays on the S-track that involved L-only cells.

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
12,23.397021,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.391955,0.256217,-0.135738,8.701443,10.519667,1.818224,4.546699,5.387942,0.841243,10,4,1,3,NeuronType.PYRAMIDAL
21,227.397021,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,1.856750,1.552593,-0.304158,16.874124,14.900123,-1.974001,9.365437,8.226358,-1.139079,19,123,62,61,NeuronType.PYRAMIDAL
48,217.397021,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.456971,0.399432,-0.057539,8.140116,NaN,NaN,4.298543,NaN,NaN,46,5,5,0,NeuronType.PYRAMIDAL


In [12]:
## 2023-05-23 - Get Common (SHARED) placefields
## Goal 1: From the cells with the placefields on both tracks, compute the degree to which they remap in position and sort them according to their distance.
is_BOTH_pf_only = np.logical_and(neuron_replay_stats_df['has_short_pf'], neuron_replay_stats_df['has_long_pf']) # (63,)
BOTH_pf_only_aclus = neuron_replay_stats_df.index[is_BOTH_pf_only].to_numpy()

## NOTE: is_BOTH_pf_only is a much more stringent requirement (and a strict subset) than `is_BOTH_only`
is_BOTH_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.SHARED # (99,)
BOTH_only_aclus = neuron_replay_stats_df.index[is_BOTH_only].to_numpy()
assert BOTH_only_aclus.shape[0] >= BOTH_pf_only_aclus.shape[0]

BOTH_pf_only_df = neuron_replay_stats_df[is_BOTH_pf_only].copy()
BOTH_pf_only_df['long_short_pf_peak_x_displacement'] = BOTH_pf_only_df['long_pf_peak_x'].values - BOTH_pf_only_df['short_pf_peak_x'].values
BOTH_pf_only_df['long_short_pf_peak_x_distance'] = BOTH_pf_only_df['long_short_pf_peak_x_displacement'].abs()
BOTH_pf_only_df.sort_values(by=['long_short_pf_peak_x_distance'], inplace=True, ascending=False)
BOTH_pf_only_df

#TODO 2023-05-23 - Can do more detailed peaks analysis with: long_results.RatemapPeaksAnalysis and short_results.RatemapPeaksAnalysis



,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,...,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type,long_short_pf_peak_x_displacement,long_short_pf_peak_x_distance
15,53.397021,True,219.397021,True,False,SplitPartitionMembership.SHARED,1.362429,1.440596,0.078168,8.931578,...,5.147003,4.815417,-0.331587,13,54,28,26,NeuronType.PYRAMIDAL,-166.0,166.0
64,233.397021,True,103.397021,True,False,SplitPartitionMembership.SHARED,2.825995,5.278539,2.452543,19.461425,...,11.143710,12.285477,1.141766,62,217,84,133,NeuronType.PYRAMIDAL,130.0,130.0
6,213.397021,True,169.397021,True,False,SplitPartitionMembership.SHARED,1.313865,0.960528,-0.353337,14.421421,...,7.867643,8.311451,0.443808,4,210,82,128,NeuronType.PYRAMIDAL,44.0,44.0
63,47.397021,True,85.397021,True,False,SplitPartitionMembership.SHARED,1.708625,0.727221,-0.981404,14.368023,...,8.038324,8.629641,0.591317,61,135,51,84,NeuronType.PYRAMIDAL,-38.0,38.0
41,45.397021,True,83.397021,True,False,SplitPartitionMembership.SHARED,0.862529,0.915665,0.053136,11.802005,...,6.332267,6.867174,0.534906,39,81,27,54,NeuronType.PYRAMIDAL,-38.0,38.0
60,63.397021,True,99.397021,True,False,SplitPartitionMembership.SHARED,5.433974,5.695663,0.261690,27.908526,...,16.671250,11.519950,-5.151299,58,218,96,122,NeuronType.PYRAMIDAL,-36.0,36.0
56,215.397021,True,179.397021,True,False,SplitPartitionMembership.SHARED,3.027875,1.725534,-1.302340,12.864660,...,7.946267,5.854188,-2.092079,54,73,46,27,NeuronType.PYRAMIDAL,36.0,36.0
44,233.397021,True,197.397021,True,False,SplitPartitionMembership.SHARED,0.611479,1.448680,0.837201,11.179379,...,5.895429,12.225815,6.330386,42,100,16,84,NeuronType.PYRAMIDAL,36.0,36.0
43,237.397021,True,201.397021,True,False,SplitPartitionMembership.SHARED,0.648754,0.490313,-0.158442,8.967668,...,4.808211,2.850531,-1.957680,41,15,8,7,NeuronType.PYRAMIDAL,36.0,36.0
39,231.397021,True,195.397021,True,False,SplitPartitionMembership.SHARED,1.718047,2.193901,0.475855,19.294147,...,10.506097,12.410463,1.904366,37,281,83,198,NeuronType.PYRAMIDAL,36.0,36.0


In [13]:
# 2023-06-19 - Look at the Long and Short only aclus and find replays in which they replay
# L_only_aclus, S_only_aclus

# long_replays

exclusive_aclus = np.hstack((L_only_aclus, S_only_aclus)) # [  2,   8,  19,  28,  36,  54,  63,  68,  73,  74, 109,   4,  13,  21,  23,  52,  58,  67,  70,  81]
exclusive_aclus
# find the cells that fire in each replay. I think this is already computed/used somewhere


array([12, 21, 48, 18, 19, 65], dtype=int64)

In [28]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _prepare_spikes_df_from_filter_epochs
# approach copied from `pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations._epoch_unit_avg_firing_rates`
from pyphocorehelpers.indexing_helpers import partition

# included_neuron_ids = deepcopy(exclusive_aclus)
included_neuron_ids = None
spikes_df: pd.DataFrame = deepcopy(curr_active_pipeline.sess.spikes_df).spikes.sliced_by_neuron_type('pyr')
# filter_epochs = deepcopy(curr_active_pipeline.sess.replay)
# spikes_df = deepcopy(long_results_obj.spikes_df) # LeaveOneOutDecodingAnalysisResult
# spikes_df[np.isin(spikes_df.aclu, included_neuron_ids)]
filter_epochs = deepcopy(long_results_obj.active_filter_epochs)

if included_neuron_ids is None:
	included_neuron_ids = spikes_df.spikes.neuron_ids

if isinstance(filter_epochs, pd.DataFrame):
	filter_epochs_df = filter_epochs
else:
	filter_epochs_df = filter_epochs.to_dataframe()
	
filter_epoch_spikes_df = deepcopy(spikes_df)
filter_epoch_spikes_df = _prepare_spikes_df_from_filter_epochs(filter_epoch_spikes_df, filter_epochs=filter_epochs_df, included_neuron_ids=included_neuron_ids, epoch_id_key_name='replay_epoch_id', debug_print=False) # replay_epoch_id

unique_replay_epoch_id_values, epoch_split_spikes_df_list = partition(filter_epoch_spikes_df, 'replay_epoch_id')
# filter_epoch_spikes_df.groupby('replay_epoch_id')
unique_replay_epoch_id_values # (198,)
epoch_spikes_unique_aclus_list = []
epoch_contains_any_exclusive_aclus = []

for an_epoch_id, epoch_specific_spikes_df in zip(unique_replay_epoch_id_values, epoch_split_spikes_df_list):
    # Loop through the epochs
	# find epochs containing at least one in `exclusive_aclus`:
	epoch_spikes_unique_aclus = epoch_specific_spikes_df.aclu.unique()
	epoch_spikes_unique_aclus_list.append(epoch_spikes_unique_aclus)
	# epoch_spikes_unique_aclus # finds lists of all the active aclus in the given epoch
	if len(epoch_spikes_unique_aclus) > 0:	
		epoch_contains_any_exclusive_aclus.append(np.isin(epoch_spikes_unique_aclus, exclusive_aclus).any())
	else:
		epoch_contains_any_exclusive_aclus.append(False)
	# epoch_contains_any_exclusive_aclus # returns True if the epoch contains one ore more spikes from the search list `exclusive_aclus`

assert len(epoch_contains_any_exclusive_aclus) == np.shape(filter_epochs_df)[0]
filter_epochs_df['contains_one_exclusive_aclu'] = epoch_contains_any_exclusive_aclus # adds the appropriate column to the filter_epochs_df
filter_epochs_df



,start,stop,label,duration,score,velocity,intercept,speed,score_LONG,velocity_LONG,intercept_LONG,speed_LONG,score_SHORT,velocity_SHORT,intercept_SHORT,speed_SHORT,contains_one_exclusive_aclu
0,49.107308,49.285484,0,0.178176,0.473504,139.775115,14.282968,139.775115,0.473504,139.775115,14.282968,139.775115,0.394694,-75.746823,144.371176,75.746823,True
1,68.571951,68.764442,1,0.192491,0.481259,-76.113799,215.237778,76.113799,0.481259,-76.113799,215.237778,76.113799,0.389931,-405.081114,233.286214,405.081114,True
2,91.226833,91.413549,2,0.186716,0.479736,-176.234177,215.701002,176.234177,0.479736,-176.234177,215.701002,176.234177,0.469504,-174.763795,170.367746,174.763795,False
3,235.576353,235.756188,3,0.179835,0.514614,-309.395869,168.113304,309.395869,0.514614,-309.395869,168.113304,309.395869,0.640654,-225.084695,158.472326,225.084695,True
4,272.462145,272.641058,4,0.178913,0.487484,143.145623,195.675563,143.145623,0.487484,143.145623,195.675563,143.145623,0.458075,276.852686,157.437154,276.852686,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,2503.348263,2503.557405,128,0.209142,0.498792,275.305437,167.463235,275.305437,0.498792,275.305437,167.463235,275.305437,0.506303,131.875685,154.790445,131.875685,True
129,2520.181371,2520.376197,129,0.194826,0.743931,-210.206581,204.239042,210.206581,0.743931,-210.206581,204.239042,210.206581,0.601656,16.137238,164.430661,16.137238,True
130,2525.535096,2525.740674,130,0.205578,0.334451,14.879566,31.495167,14.879566,0.334451,14.879566,31.495167,14.879566,0.519752,453.761591,50.101389,453.761591,False
131,2530.724715,2530.918773,131,0.194058,0.706966,255.289901,20.031194,255.289901,0.706966,255.289901,20.031194,255.289901,0.983988,162.225189,54.724223,162.225189,True


In [ ]:
# ## Non-for-loop version:
# # find epochs containing at least one in `exclusive_aclus`:
# epoch_spikes_unique_aclus = [epoch_split_spikes_df.aclu.unique() for epoch_split_spikes_df in epoch_split_spikes_df_list]
# # epoch_spikes_unique_aclus # finds lists of all the active aclus in the given epoch
# epoch_contains_any_exclusive_aclus = [np.isin(epoch_aclus, exclusive_aclus).any() for epoch_aclus in epoch_spikes_unique_aclus]
# # epoch_contains_any_exclusive_aclus # returns True if the epoch contains one ore more spikes from the search list `exclusive_aclus`
# filter_epochs['contains_one_exclusive_aclu'] = epoch_contains_any_exclusive_aclus
# # epoch_contains_any_exclusive_aclus

In [ ]:
# first slice only to the neurons of interest:
# filter_epoch_spikes_df


In [ ]:
""" 24 cells, 133 Epochs, 789 Total Timebins
# Other
	
	- is_non_firing_time_bin: numpy.ndarray - (24, 789)
	- all_epochs_num_epoch_time_bins: numpy.ndarray - (133,)
	
    ## Indexing Helpers:
		- all_epochs_reverse_flat_epoch_indicies_array: numpy.ndarray - (789,)
		- split_by_epoch_reverse_flattened_time_bin_indicies: list - (133,)
    
	- original_1D_decoder: pyphoplacecellanalysis.Analysis.Decoder.reconstruction.BasePositionDecoder
		- pf: neuropy.analyses.placefields.PfND
		- neuron_IDXs: numpy.ndarray - (24,)
		- neuron_IDs: numpy.ndarray - (24,)
		- F: numpy.ndarray - (119, 24)
		- P_x: numpy.ndarray - (119, 1)
        
	- flat_all_epochs_decoded_epoch_time_bins: numpy.ndarray - (24, 789)

# Measured
	- flat_all_epochs_measured_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - (24, 789)


# Expected
	- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_firing_rates: numpy.ndarray - (24, 789)
    
    
    
## Epoch-based
	- all_epochs_decoded_epoch_time_bins_mean: numpy.ndarray - (133, 24)

    
    
    
- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - .shape: (24, 789) 
- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - .shape: (24, 789)


"""

# 2023-04-13 - Find Good looking epochs:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
import matplotlib.patheffects as path_effects

from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

curr_results_obj = long_results_obj
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_pagination_controller = DecodedEpochSlicesPaginatedFigureController.init_from_decoder_data(curr_results_obj.active_filter_epochs, curr_results_obj.all_included_filter_epochs_decoder_result, 
	xbin=curr_results_obj.original_1D_decoder.xbin, global_pos_df=global_session.position.df, a_name='TestDecodedEpochSlicesPaginationController', active_context=active_identifying_session_ctx,  max_subplots_per_page=200) # 10
# _out_pagination_controller

In [10]:
## Getting selections out of DecoedEpochSlicesPaginationController
curr_active_pipeline.display_output_history_list
# [IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'long_only', '[2, 8, 19, 28, 36, 54, 63, 68, 73, 74, 109]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'short_only', '[4, 13, 21, 23, 52, 58, 67, 70, 81]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '1of9', '[3, 5, 6, 7, 9, 10, 11, 12, 14, 15]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '2of9', '[16, 17, 18, 20, 22, 24, 25, 26, 27, 29]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '3of9', '[30, 31, 32, 33, 34, 35, 37, 38, 39, 40]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '4of9', '[41, 42, 43, 44, 45, 46, 47, 48, 49, 50]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '5of9', '[51, 53, 55, 56, 57, 59, 60, 61, 62, 64]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '6of9', '[65, 66, 69, 71, 72, 75, 76, 77, 78, 79]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '7of9', '[80, 82, 83, 84, 85, 86, 87, 88, 89, 90]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '8of9', '[91, 92, 93, 94, 95, 96, 97, 98, 99, 100]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '9of9', '[101, 102, 103, 104, 105, 106, 107, 108]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_1d_placefields')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_1d_placefields')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_1d_placefields')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_1d_placefield_validations')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_1d_placefield_validations')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_1d_placefield_validations')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_2d_placefield_result_plot_ratemaps_2D')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_2d_placefield_result_plot_ratemaps_2D')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_2d_placefield_result_plot_ratemaps_2D')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_2d_placefield_occupancy')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_2d_placefield_occupancy')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_2d_placefield_occupancy')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_short_long_firing_rate_index_comparison')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_short_expected_v_observed_firing_rate')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_grid_bin_bounds_validation')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_running_and_replay_speeds_over_time')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_short_laps')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_and_short_firing_rate_replays_v_laps')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_short_long_pf1D_comparison')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_and_short_stacked_epoch_slices')>]

# Ideally could search like: '_display_long_and_short_stacked_epoch_slices'
active_result_backup = curr_active_pipeline.last_added_display_output

In [11]:
list(active_result_backup.keys()) # ['name', 'context', 'figures', 'axes', 'plot_data']

active_result_backup['context'] # (IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'DecodedEpochSlices', 'replays', 'long_results_obj')>,
								#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'DecodedEpochSlices', 'replays', 'short_results_obj')>)

pagination_controllers = active_result_backup.plot_data['controllers']

In [12]:
long_controller = pagination_controllers[0]
short_controller = pagination_controllers[1]

In [32]:
long_controller.selected_indicies # array([ 13,  14,  15,  25,  27,  28,  31,  37,  42,  45,  48,  57,  61,  62,  63,  76,  79,  82,  89,  90, 111, 112, 113, 115])

# np.array([5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63, 64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95, 104, 105, 122, 123])

array([  5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63,
        64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95,
       104, 105, 122, 123])

In [31]:
short_controller.selected_indicies # array([  9,  11,  13,  14,  15,  20,  22,  25,  37,  40,  45,  48,  61, 62,  76,  79,  84,  89,  90,  93,  94, 111, 112, 113, 115, 121])

# np.array([ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49, 54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100, 104, 105, 111, 114, 115, 117, 118, 122, 123, 131])

array([ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49,
        54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100,
       104, 105, 111, 114, 115, 117, 118, 122, 123, 131])

In [35]:
assert long_controller.is_selected.shape[0] == long_results_obj.active_filter_epochs.n_epochs
assert short_controller.is_selected.shape[0] == short_results_obj.active_filter_epochs.n_epochs
# for some reason the active_filter_epochs on both the long and the short objects contain properties about the other one as well, so keep with tradition and add them to both as well
long_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
long_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected
short_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
short_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected

In [ ]:
long_controller.is_selected, long_controller.selected_indicies, long_controller.plots_data.epoch_slices, long_controller.params.flat_all_data_indicies

In [ ]:
print(list(long_controller.params.keys())) # ['name', 'window_title', 'num_slices', '_debug_test_max_num_slices', 'active_num_slices', 'global_epoch_start_t', 'global_epoch_end_t', 'single_plot_fixed_height', 'all_plots_height', 'epoch_labels', 'variable_name', 'xbin', 'enable_flat_line_drawing', 'debug_print', 'active_identifying_figure_ctx', 'flat_all_data_indicies', 'is_selected', 'callback_id', 'on_render_page_callbacks']

In [23]:
long_controller.params.active_identifying_figure_ctx.adding_context(user_annotation="selections")

TypeError: adding_context() missing 1 required positional argument: 'collision_prefix'

on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =111
	selection_indicies: [ 12  13  15  17  20  24  30  31  32  33  41  43  49  54  55  68  70  71
  73  76  77  78  84  89  94 100 104 105 111]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =68
	selection_indicies: [ 5 13 15 17 20 21 24 31 33 43 44 49 66 68 70 71]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =63
	selection_indicies: [ 5 13 15 17 20 21 24 31 33 43 44 49 63 66 68 70 71]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =64
	selection_indicies: [ 5 13 15 17 20 21 24 31 33 43 44 49 63 64 66 68 70 71]


In [ ]:
long_controller.selected_epoch_times
# array([[ 292.62367098,  292.80826736],
#        [ 528.48329294,  528.68561475],
#        [ 572.24842637,  572.44236163],
#        [ 677.99306343,  678.25522777],
#        [ 802.91157867,  803.11436129],
#        [ 831.70962856,  831.90743454],
#        [ 888.22678107,  888.46492239],
#        [ 945.48035303,  945.68543965],
#        [ 954.00517939,  954.2546871 ],
#        [1033.05203044, 1033.22639707],
#        [1061.35727035, 1061.5943979 ],
#        [1214.61082388, 1214.834834  ],
#        [1722.78022002, 1722.95406441],
#        [1741.58530479, 1741.79413924],
#        [1743.25171034, 1743.53445707]])

In [ ]:
short_controller.selected_epoch_times
# array([[ 489.42507549,  489.65575186],
#        [ 528.48329294,  528.68561475],
#        [ 572.24842637,  572.44236163],
#        [ 677.99306343,  678.25522777],
#        [ 802.91157867,  803.11436129],
#        [ 888.22678107,  888.46492239],
#        [ 931.21227202,  931.45262518],
#        [ 945.48035303,  945.68543965],
#        [ 950.18327543,  950.44046313],
#        [ 954.00517939,  954.2546871 ],
#        [1024.83173129, 1025.10320379],
#        [1033.05203044, 1033.22639707],
#        [1214.61082388, 1214.834834  ],
#        [1326.54476977, 1326.76506278],
#        [1329.84606216, 1330.0684134 ],
#        [1732.09310613, 1732.29060491],
#        [1741.58530479, 1741.79413924],
#        [1743.25171034, 1743.53445707],
#        [1751.03108555, 1751.22059714],
#        [1769.90424581, 1770.10497019],
#        [1776.56793264, 1776.78871716],
#        [1871.00426701, 1871.21629634],
#        [1917.09385877, 1917.2703758 ],
#        [1994.07242543, 1994.2525367 ],
#        [2038.53127755, 2038.70899266],
#        [2064.36693792, 2064.54016792],
#        [2111.52742834, 2111.80120484],
#        [2125.54830564, 2125.73772506]])

In [18]:
long_results_obj.active_filter_epochs.n_epochs # 122

133

# TODO 2023-06-19 14:31: - [ ] Come up with system for storing user-annotations:
 Need to store user-annotations and labeled selections for Epochs:

	date_annotated: datetime - the date the annotations were made
	annotation_author: str - the person who made the annotations
	annotation_content: Any - the content of the annotation



In [30]:
# All epochs are shown for both decoders. So the result should be an epochs table with separate columns for the inclusion in each decoder.
@define(slots=False, eq=False)
class EpochSelectionSet:
    selected_indicies: np.ndarray # a list of the selected inidices

long_selection = EpochSelectionSet(selected_indicies=np.array([5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63, 64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95, 104, 105, 122, 123]))
short_selection = EpochSelectionSet(selected_indicies=np.array([ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49, 54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100, 104, 105, 111, 114, 115, 117, 118, 122, 123, 131]))

In [31]:

filter_epochs_df['long_is_user_included'] = np.isin(filter_epochs_df.index, long_selection.selected_indicies)
filter_epochs_df['short_is_user_included'] = np.isin(filter_epochs_df.index, short_selection.selected_indicies)

## Finally filter to find the potential example epochs:

# only include those with one or more exclusive aclu:
considered_filter_epochs_df = filter_epochs_df[filter_epochs_df.contains_one_exclusive_aclu].copy()
# require inclusion for long or short:
considered_filter_epochs_df = considered_filter_epochs_df[np.logical_xor(filter_epochs_df['long_is_user_included'], filter_epochs_df['short_is_user_included'])]
considered_filter_epochs_df



,start,stop,label,duration,score,velocity,intercept,speed,score_LONG,velocity_LONG,intercept_LONG,speed_LONG,score_SHORT,velocity_SHORT,intercept_SHORT,speed_SHORT,contains_one_exclusive_aclu,long_is_user_included,short_is_user_included
0,49.107308,49.285484,0,0.178176,0.473504,139.775115,14.282968,139.775115,0.473504,139.775115,14.282968,139.775115,0.394694,-75.746823,144.371176,75.746823,True,False,False
1,68.571951,68.764442,1,0.192491,0.481259,-76.113799,215.237778,76.113799,0.481259,-76.113799,215.237778,76.113799,0.389931,-405.081114,233.286214,405.081114,True,False,False
2,91.226833,91.413549,2,0.186716,0.479736,-176.234177,215.701002,176.234177,0.479736,-176.234177,215.701002,176.234177,0.469504,-174.763795,170.367746,174.763795,False,False,False
3,235.576353,235.756188,3,0.179835,0.514614,-309.395869,168.113304,309.395869,0.514614,-309.395869,168.113304,309.395869,0.640654,-225.084695,158.472326,225.084695,True,False,False
4,272.462145,272.641058,4,0.178913,0.487484,143.145623,195.675563,143.145623,0.487484,143.145623,195.675563,143.145623,0.458075,276.852686,157.437154,276.852686,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,2503.348263,2503.557405,128,0.209142,0.498792,275.305437,167.463235,275.305437,0.498792,275.305437,167.463235,275.305437,0.506303,131.875685,154.790445,131.875685,True,False,False
129,2520.181371,2520.376197,129,0.194826,0.743931,-210.206581,204.239042,210.206581,0.743931,-210.206581,204.239042,210.206581,0.601656,16.137238,164.430661,16.137238,True,False,False
130,2525.535096,2525.740674,130,0.205578,0.334451,14.879566,31.495167,14.879566,0.334451,14.879566,31.495167,14.879566,0.519752,453.761591,50.101389,453.761591,False,False,False
131,2530.724715,2530.918773,131,0.194058,0.706966,255.289901,20.031194,255.289901,0.706966,255.289901,20.031194,255.289901,0.983988,162.225189,54.724223,162.225189,True,False,True


In [ ]:
assert long_controller.is_selected.shape[0] == long_results_obj.active_filter_epochs.n_epochs
assert short_controller.is_selected.shape[0] == short_results_obj.active_filter_epochs.n_epochs
# for some reason the active_filter_epochs on both the long and the short objects contain properties about the other one as well, so keep with tradition and add them to both as well
long_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
long_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected
short_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
short_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected

# Other Programmatic Figures

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

## Builds proper sort indicies for '_display_short_long_pf1D_comparison'
long_ratemap = long_pf1D.ratemap
short_ratemap = short_pf1D.ratemap

curr_any_context_neurons = _find_any_context_neurons(*[k.neuron_ids for k in [long_ratemap, short_ratemap]])
curr_any_context_neurons

In [ ]:
from neuropy.plotting.ratemaps import plot_ratemap_1D
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_short_v_long_pf1D_comparison, LongShortTrackComparingDisplayFunctions

curr_active_pipeline.reload_default_display_functions()

%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None
# sort_idx = None
# sort_idx = sort_ind
# sort_idx = sort_indicies
# sort_idx = new_all_aclus_sort_indicies
sort_idx = 'peak_long'
# sort_idx = 'bad'

# out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=True, fignum='Short v Long pf1D Comparison',
#                                    long_kwargs={'included_unit_neuron_IDs': curr_any_context_neurons, 'included_unit_indicies': None, 'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
#                                    short_kwargs={'included_unit_neuron_IDs': curr_any_context_neurons, 'included_unit_indicies': None,'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
#                                   )

out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )
                                  

# ax = out.axes[0]

In [ ]:
curr_active_pipeline.display('_display_short_long_pf1D_comparison', single_figure=False)

# 2023-06-01 - Testing new metrics:

In [ ]:

curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
## Extract variables from results object:
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

print(f'long_test: {np.mean(long_long_diff)}')
print(f'short_test: {np.mean(short_short_diff)}')

In [ ]:
long_session.replay

In [ ]:
# long_results_obj.original_1D_decoder
replay_result_df = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())
replay_result_df

In [ ]:
short_results_obj.active_filter_epochs.to_dataframe()

In [ ]:
# Look at lap speed over time - doesn't look like there's a difference between the long and the short track speeds
global_session.position.compute_higher_order_derivatives()
running_pos_df = global_session.position.to_dataframe()
ax = running_pos_df.plot(x='t', y=['lin_pos','speed'], title='Running Speed over Time')
ax

In [ ]:
# Call the function
fig, ax = plot_lines(replay_result_df)

In [ ]:
import matplotlib
%matplotlib qt

curr_active_pipeline.reload_default_display_functions()
out_man = curr_active_pipeline.get_output_manager()
out_man.get_figure_output_path
curr_active_pipeline.write_figure_to_output_path



In [ ]:
_out = curr_active_pipeline.display('_display_grid_bin_bounds_validation')
# _display_running_and_replay_speeds_over_time


# 2023-06-13 11:01: - [ ] New display function testing

In [ ]:
import matplotlib as mpl
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path, programmatic_display_to_PDF
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_pdf_metadata_from_display_context # newer version of build_pdf_export_metadata
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

%matplotlib qt

from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_render_to_file

In [ ]:
curr_active_pipeline.reload_default_display_functions()


In [ ]:
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap, visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_raster_plot # used in `plot_kourosh_activity_style_figure`
import pyphoplacecellanalysis.External.pyqtgraph as pg # used in `plot_kourosh_activity_style_figure`

heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(long_pf1D.ratemap.normalized_tuning_curves, title=f"pf1D Visualization")
# Apply the colormap
# heatmap_pf1D_img.setLookupTable(lut)
heatmap_pf1D_win.show()

In [36]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _prepare_spikes_df_from_filter_epochs

long_ratemap = long_pf1D.ratemap
short_ratemap = short_pf1D.ratemap
curr_any_context_neurons = _find_any_context_neurons(*[k.neuron_ids for k in [long_ratemap, short_ratemap]])

# filter_epochs = deepcopy(long_replays)

filter_epochs = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())
filter_epoch_spikes_df = _prepare_spikes_df_from_filter_epochs(long_session.spikes_df, filter_epochs=filter_epochs, included_neuron_ids=curr_any_context_neurons, epoch_id_key_name='replay_epoch_id', debug_print=False) # replay_epoch_id
filter_epoch_spikes_df

,t,t_seconds,t_rel_seconds,shank,cluster,aclu,qclu,x,y,speed,...,cell_type,flat_spike_idx,x_loaded,y_loaded,lin_pos,fragile_linear_neuron_IDX,PBE_id,scISI,neuron_IDX,replay_epoch_id
21031,1598546.0,525499.204699,49.107308,1,9,7,2,38.488692,153.887024,1.852818,...,NeuronType.PYRAMIDAL,21031,0.151364,0.528108,36.231171,2,0,0.258232,2,0
21055,1599972.0,525499.248506,49.151115,2,34,27,2,38.297371,153.904587,5.116750,...,NeuronType.PYRAMIDAL,21055,0.151044,0.529029,36.100123,15,0,7.160767,15,0
21062,1600256.0,525499.257230,49.159839,2,18,20,4,38.236352,153.895181,5.686409,...,NeuronType.PYRAMIDAL,21062,0.151080,0.529312,36.065503,10,0,1.744281,10,0
21064,1600317.0,525499.259104,49.161713,1,9,7,2,38.223246,153.893161,5.808765,...,NeuronType.PYRAMIDAL,21064,0.151087,0.529373,36.058067,2,0,0.054405,2,0
21082,1601997.0,525499.310714,49.213323,8,2,45,2,37.951220,153.824898,4.754838,...,NeuronType.PYRAMIDAL,21082,0.151543,0.531110,35.913419,25,0,0.650864,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483896,39928340.0,526676.695368,1226.597977,12,4,64,4,228.201262,145.150455,2.557825,...,NeuronType.PYRAMIDAL,483896,0.816330,0.497681,229.082277,37,150,0.144292,37,50
483900,39928744.0,526676.707779,1226.610388,7,11,44,4,228.226482,145.180508,2.993425,...,NeuronType.PYRAMIDAL,483900,0.816547,0.497688,229.105888,24,150,0.718971,24,50
483908,39930110.0,526676.749742,1226.652351,8,25,56,2,228.281766,145.243598,2.181934,...,NeuronType.PYRAMIDAL,483908,0.817280,0.497712,229.175608,33,150,0.066847,33,50
483912,39930368.0,526676.757668,1226.660277,3,12,34,4,228.277974,145.241882,1.487413,...,NeuronType.PYRAMIDAL,483912,0.817420,0.497717,229.171949,18,150,0.076585,18,50


In [37]:
from pyphocorehelpers.indexing_helpers import partition

unique_replay_epoch_id_values, epoch_split_spikes_df_list = partition(filter_epoch_spikes_df, 'replay_epoch_id')
# filter_epoch_spikes_df.groupby('replay_epoch_id')
unique_replay_epoch_id_values # (198,)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
      dtype=int64)

In [ ]:
len(epoch_split_spikes_df_list)

In [ ]:
len(epoch_spikes_unique_aclus)
len(epoch_contains_any_exclusive_aclus)
len(filter_epochs)

In [38]:
# find epochs containing at least one in `exclusive_aclus`:
epoch_spikes_unique_aclus = [epoch_split_spikes_df.aclu.unique() for epoch_split_spikes_df in epoch_split_spikes_df_list]
# epoch_spikes_unique_aclus # finds lists of all the active aclus in the given epoch
epoch_contains_any_exclusive_aclus = [np.isin(epoch_aclus, exclusive_aclus).any() for epoch_aclus in epoch_spikes_unique_aclus]
# epoch_contains_any_exclusive_aclus # returns True if the epoch contains one ore more spikes from the search list `exclusive_aclus`
filter_epochs['contains_one_exclusive_aclu'] = epoch_contains_any_exclusive_aclus
# epoch_contains_any_exclusive_aclus

NameError: name 'exclusive_aclus' is not defined

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot

app, win, plots, plots_data = plot_multiple_raster_plot(filter_epochs_df, filter_epoch_spikes_df, included_neuron_ids=shared_aclus, epoch_id_key_name='replay_epoch_id', scatter_app_name="Pho Stacked Replays")